In [2]:
import os
import re

from tqdm import tqdm

from bs4 import BeautifulSoup
import pandas as pd

# Настройки

In [ ]:
HTML_DIR = '/dataset/'

# Загрузка исходных данных

### Парсинг html-файлов

In [ ]:
def parse_CV(file):
    
    soup = BeautifulSoup(open(file), 'html.parser')
    features_dict = {}

    sex = soup.find_all('div', class_ = '')
    features_dict['Sex'] = [x.a.text for x in sex]

    age = soup.find_all('div', class_ = '')
    features_dict['Age'] = [int(re.findall(r'\d+', x.text)[0]) for x in age]

    post = soup.find_all('div', class_ = '')
    features_dict['Post'] = [x.a.text for x in post]

    education = soup.find_all('div', class_ = '')
    features_dict['Education'] = [x.a.text for x in education]

    experience = soup.find_all('div', class_ = '')
    features_dict['Experience'] = [x.a.text for x in experience]

    return features_dict    

In [ ]:
df = pd.DataFrame()

for i, file in tqdm(enumerate(os.listdir(HTML_DIR))):
    df.iloc[i] = parse_CV(file)

In [ ]:
df

# Обработка датасета